# Combining topologies from PDB and SDF with OpenFF toolkit

In [1]:
from openff.toolkit import Molecule, Topology
from openmm import LangevinMiddleIntegrator
from openmm.unit import *
from openmm import app

ligand = Molecule.from_file("ligand.sdf")
receptor_path = 'receptor.pdb'

top = Topology.from_pdb(receptor_path)
n_protein = top.n_atoms

### Combine topologies

In [2]:
top.add_molecule(ligand)
n_total = top.n_atoms

Visualize the complex to check that it combined OK

In [3]:
w = top.visualize()
w.clear_representations()
w.add_representation(
    "licorice",
    radius=0.1,
    selection=[*range(n_protein)],
)
w.add_representation(
    "spacefill",
    selection=[*range(n_protein, n_total)],
)
w

NGLWidget()

## Define force fields

In [4]:
forcefield = app.ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3p.xml')

GAFF generator for ligand...

In [5]:
from openmmforcefields.generators import GAFFTemplateGenerator

gaff = GAFFTemplateGenerator(molecules=ligand)
forcefield.registerTemplateGenerator(gaff.generator)

OR Smirnoff generator

In [9]:
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

smirnoff = SMIRNOFFTemplateGenerator(molecules=ligand)
forcefield.registerTemplateGenerator(smirnoff.generator)

## Parametrize ligand with the given forcefield

In [6]:
%%time
system = forcefield.createSystem(top.to_openmm())
print("done")

done
CPU times: user 898 ms, sys: 15.3 ms, total: 913 ms
Wall time: 3min 56s


## Create simulation and assign coordinates

In [7]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
openmm_simulation = app.Simulation(top.to_openmm(), system, integrator)
openmm_simulation.context.setPositions(top.get_positions().magnitude)